# Продвинутое машинное обучение: 
Домашнее задание 2

Задача: построить вероятностную рейтинг-систему для спортивного “Что? Где? Когда?” (ЧГК).

https://rating.chgk.info/


In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score

from scipy.stats import spearmanr, kendalltau

from tqdm import tqdm
from os import listdir, path

In [2]:
display(sorted(listdir('./data')))

['player_results.csv',
 'player_results_X.csv',
 'players-release-2020-01-02.csv',
 'players-release-2020-12-25.csv',
 'players.pkl',
 'results.pkl',
 'team_results.csv',
 'team_results_X.csv',
 'tournaments.csv',
 'tournaments.pkl']

# Ввод и анализ данных

1.Прочитайте и проанализируйте данные, выберите турниры,в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl).

In [3]:
with open('./data/tournaments.pkl', 'rb') as f:
    raw_tournaments = pd.read_pickle(f)
    
with open('./data/results.pkl', 'rb') as f:
    raw_results = pd.read_pickle(f)
    
display(f'Raw tournaments: {len(raw_tournaments)}')
display(f'Raw results:     {len(raw_results)}')

'Raw tournaments: 5528'

'Raw results:     5528'

In [4]:
def parse_tournaments(tournaments, dateStart):
    columns = [
        'id',
        'name',
        'type',
        'typeId',
        'dateStart',
        'dateEnd',
        'questionsTotal',
    ]
    
    data = []
    for tournament in tqdm(tournaments.values()):
        if dateStart and tournament['dateStart'] > dateStart:
            data.append([
                tournament['id'],
                tournament['name'],
                tournament['type']['name'] if tournament['type'] else np.nan,
                tournament['type']['id'] if tournament['type'] else np.nan,
                tournament['dateStart'],
                tournament['dateEnd'],
                sum(tournament['questionQty'].values()),
            ])
    return pd.DataFrame(data, columns=columns)

In [5]:
tournaments = parse_tournaments(raw_tournaments, '2019')
del raw_tournaments

display(tournaments.head(2))
display(tournaments.shape)
tournaments.describe(include='all').T.fillna('')

100%|██████████| 5528/5528 [00:00<00:00, 383628.33it/s]


,id,name,type,typeId,dateStart,dateEnd,questionsTotal
0,4628,Семь сорок,Синхрон,3,2020-12-30T16:00:00+03:00,2020-12-30T16:00:00+03:00,36
1,4772,Синхрон северных стран. Зимний выпуск,Синхрон,3,2019-01-05T19:00:00+03:00,2019-01-09T19:00:00+03:00,36


(1109, 7)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,1109.0,,,,5853.67358,373.345899,4628.0,5565.0,5860.0,6168.0,6485.0
name,1109.0,938,"Онлайн: 19:00 Не числом, а умением - 2 (NEW!)",6,,,,,,,
type,1109.0,5,Синхрон,556,,,,,,,
typeId,1109.0,,,,3.148783,1.709089,2.0,2.0,3.0,3.0,8.0
dateStart,1109.0,823,2019-02-22T20:00:00+03:00,7,,,,,,,
dateEnd,1109.0,851,2020-12-30T14:00:00+03:00,8,,,,,,,
questionsTotal,1109.0,,,,47.307484,33.140579,3.0,36.0,36.0,45.0,500.0


In [6]:
def team_results(results, tournamentIds):
    for idx in tournamentIds:
        rec = results[idx]
        for team in rec:
            yield idx, team

def parse_results(results, tournamentIds):
    columns = [
        'tournamentId',
        'teamId',
        'teamName',
        'position',
        'playerId',
        'playerName',
        'mask',
        'answersTotal',
        'maskLen',
        'maskSum',
    ]
    
    data = []
    for idx, team in tqdm(team_results(results, tournamentIds)):
        # пропускаем результаты без повопросного описания
        if 'mask' not in team or not team['mask']:
            continue
        mask = team['mask']

        # пропускаем результаты со снятыми вопросами
        if 'X' in mask:
            continue

        for member in team['teamMembers']:
            if len(member) == 0:
                continue
            player = member['player']

            data.append([
                idx,
                team['team']['id'],
                team['team']['name'],
                team['position'],
                player['id'],
                ' '.join(player[k] or '' for k in ['surname', 'name', 'patronymic']),
                mask,
                team['questionsTotal'],
                len(mask),
                sum(c == '1' for c in mask)
            ])
                
    df = pd.DataFrame(data, columns=columns) \
        .set_index('tournamentId')
    
    df['maskMaxLen'] = df.groupby(by='tournamentId')['maskLen'].max()
    df.reset_index(inplace=True)
    
    # пропускаем результаты с некорректными масками
    correct_masks = (df['maskLen'] == df['maskMaxLen']) & (df['answersTotal'] == df['maskSum'])
    df = df[correct_masks][columns[:-3]].copy()

    return df

In [7]:
results = parse_results(raw_results, tournaments['id'].values)
del raw_results

display(results.head(2))
display(results.shape)
results.describe(include='all').T.fillna('')

109721it [00:01, 55086.50it/s]


,tournamentId,teamId,teamName,position,playerId,playerName,mask
0,4772,45556,Рабочее название,1.0,6212,Выменец Юрий Яковлевич,111111111011111110111111111100010010
1,4772,45556,Рабочее название,1.0,18332,Либер Александр Витальевич,111111111011111110111111111100010010


(445734, 7)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
tournamentId,445734.0,,,,5661.457596,337.865184,4772.0,5452.0,5740.0,5856.0,6456.0
teamId,445734.0,,,,51052.790588,23037.751309,2.0,44012.0,57676.0,68615.0,78494.0
teamName,445734.0,11602,МГТУ - Легионеры,1467,,,,,,,
position,445734.0,,,,180.023633,215.877514,1.0,31.0,95.5,240.5,1247.5
playerId,445734.0,,,,116555.099761,67367.636476,15.0,54537.0,124615.0,176493.0,224697.0
playerName,445734.0,61048,Мельникова Ольга Андреевна,235,,,,,,,
mask,445734.0,85238,000000000000000000000000000000000000,1001,,,,,,,


In [8]:
players = results[['playerId', 'playerName']] \
    .drop_duplicates() \
    .set_index('playerId')

display(players.shape)
players.sort_values(by='playerName')

(61588, 1)

,playerName
playerId,
189667,Ёжиков Александр
110048,Ёжикова Ирина Игоревна
189364,Ёзденир Надежда Ильинична
198877,Ёкубов Сохиб Тохирович
213996,Ёкубов Улугбек Зокиругли
...,...
215270,Ящук Леонид
191434,ван Вилген Диана
191455,ван Вилген Людмила


In [9]:
team_results = results[['tournamentId', 'teamId','teamName', 'position']] \
    .drop_duplicates() \
    .set_index('teamId') \
    .sort_values(by=['tournamentId', 'position', 'teamName']) \
    .reset_index()

display(team_results.shape)
team_results[team_results['position'] < 6].head(20)

(86628, 4)

,teamId,tournamentId,teamName,position
0,45556,4772,Рабочее название,1.0
1,58596,4772,Аутята,5.5
2,40931,4772,Здоровенный Я,5.5
3,47075,4772,Оператор Дамблдора,5.5
4,1030,4772,Сборная Бутана,5.5
5,53185,4772,Сербский мультфильм,5.5
6,68786,4772,Сцилла,5.5
7,5444,4772,Эйфью,5.5
8,4252,4772,Ять,5.5
231,69309,4973,Брют,1.5


In [10]:
questions = results[['tournamentId', 'teamId', 'position', 'playerId', 'mask']] \
    .merge(tournaments[['id', 'dateStart']], how='inner', left_on='tournamentId', right_on='id') \
    .drop(columns='id')

display(questions.head(3))
display(questions.shape)
questions.describe(include='all').T.fillna('')

,tournamentId,teamId,position,playerId,mask,dateStart
0,4772,45556,1.0,6212,111111111011111110111111111100010010,2019-01-05T19:00:00+03:00
1,4772,45556,1.0,18332,111111111011111110111111111100010010,2019-01-05T19:00:00+03:00
2,4772,45556,1.0,18036,111111111011111110111111111100010010,2019-01-05T19:00:00+03:00


(445734, 6)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
tournamentId,445734.0,,,,5661.457596,337.865184,4772.0,5452.0,5740.0,5856.0,6456.0
teamId,445734.0,,,,51052.790588,23037.751309,2.0,44012.0,57676.0,68615.0,78494.0
position,445734.0,,,,180.023633,215.877514,1.0,31.0,95.5,240.5,1247.5
playerId,445734.0,,,,116555.099761,67367.636476,15.0,54537.0,124615.0,176493.0,224697.0
mask,445734.0,85238,000000000000000000000000000000000000,1001,,,,,,,
dateStart,445734.0,583,2019-10-17T00:01:00+03:00,9693,,,,,,,


In [11]:
# Обучающая выборка - турниры за 2019 год
questions2019 = questions[questions['dateStart'] < '2020'] \
    .drop(columns=['dateStart'])

# Проверочная выборка - турниры за 2020 год
questions2020 = questions[(questions['dateStart'] > '2020')& (questions['dateStart'] < '2021')] \
    .drop(columns=['dateStart'])

display((questions2019.shape, questions2020.shape))
questions2019[::10000].head()

((350475, 5), (95259, 5))

,tournamentId,teamId,position,playerId,mask
0,4772,45556,1.0,6212,111111111011111110111111111100010010
10000,4974,68299,631.0,131752,000111011111100000010000101000011000
20000,5010,61705,10.5,172973,111111110001001011011010101111110111
30000,5025,6088,134.0,20096,111100001001111101000001010100001001000010000000
40000,5071,62365,247.0,166831,100101100001011100010110100010101111


# Baseline-модель

2.Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков на основе повопросных результатов

In [12]:
# Повопросные результаты каждого игрока
def explode_mask(df):
    df = df.copy()
    display('splitting mask...')
    df['mask'] = df['mask'].apply(lambda x: [(i+1, int(c == '1')) for i, c in enumerate(x)])
    
    display('exploding mask...')
    df = df.explode('mask')
    
    display('making ids...')
    df['questionId'] = df['tournamentId'].apply(str)
    df['questionId'] += df['mask'].apply(lambda x: '#' + str(x[0]))
    df['answer'] = df['mask'].apply(lambda x: x[1])
    df.drop(columns=['mask'], inplace=True)
    return df

train = explode_mask(questions2019)
test = explode_mask(questions2020)

display((train.shape, test.shape))
train[::1000000].head()

'splitting mask...'

'exploding mask...'

'making ids...'

'splitting mask...'

'exploding mask...'

'making ids...'

((14782108, 6), (3761558, 6))

,tournamentId,teamId,position,playerId,questionId,answer
0,4772,45556,1.0,6212,4772#1,1
27777,5021,66711,341.5,140388,5021#29,0
52027,5110,53341,61.5,26261,5110#34,1
77441,5331,6936,39.5,32004,5331#8,0
102204,5425,69349,25.5,115510,5425#18,0


In [13]:
X_train = train[['playerId', 'questionId']]
X_test = test[['playerId', 'questionId']]

y_train = train['answer']
y_test = test['answer']

display(f'Train: {(X_train.shape, y_train.shape)}')
display(f'Test:  {(X_test.shape, y_test.shape)}')

'Train: ((14782108, 2), (14782108,))'

'Test:  ((3761558, 2), (3761558,))'

In [14]:
class ChgkRatingBaseModel(object):
    def __init__(self, verbose=False):
        self.TOP_RATING = 14000
        self.encoder = OneHotEncoder(handle_unknown='ignore')
        self.model = LogisticRegression(
            solver='sag', penalty='l2', random_state=1, n_jobs=-1
        )
        self.verbose = verbose
        
    def fit(self, X, y):
        if self.verbose:
            print(f'encoding...\ninput X: {X.shape}')
        X_enc = self.encoder.fit_transform(X)
        if self.verbose:
            print(f'encoded: {X_enc.shape}\nfitting...')
        self.model.fit(X_enc, y)
        if self.verbose:
            print(f'done')
        return self
    
    def predict(self, X):
        if self.verbose:
            print(f'encoding...\ninput X: {X.shape}')
        X_enc = self.encoder.transform(X)
        if self.verbose:
            print(f'encoded X: {X_enc.shape}\npredicting...')
        y = self.model.predict(X_enc)
        if self.verbose:
            print(f'done')
        return y
    
    def predict_proba(self, X):
        if self.verbose:
            print(f'encoding...\ninput X: {X.shape}')
        X_enc = self.encoder.transform(X)
        if self.verbose:
            print(f'encoded: {X_enc.shape}\npredicting...')
        y = self.model.predict_proba(X_enc)
        if self.verbose:
            print(f'done')
        return y
    
    def player_rating(self):
        num_players = len(self.encoder.categories_[0])
        data = self.model.coef_[0, :num_players]
        index = self.encoder.categories_[0]
        # отрицательный и нулевой рейтинг это плохо, применим сигмоиду
        rating = pd.Series(self.TOP_RATING * (1 - 1/(1+np.exp(data))), index=index)
        rating.index.name = 'playerId'
        rating.name='playerRating'
        rating.sort_values(ascending=False,inplace=True)
        return rating
        
    def question_rating(self):
        num_players = len(self.encoder.categories_[0])
        data = self.model.coef_[0, num_players:]
        index = self.encoder.categories_[1]
        rating = pd.Series(self.TOP_RATING * (1 - 1/(1+np.exp(data))), index=index)
        rating.index.name = 'questionId'
        rating.name='questionRating'
        rating.sort_values(inplace=True)
        return rating

In [15]:
base_model = ChgkRatingBaseModel(verbose=True)
base_model.fit(X_train, y_train)

encoding...
input X: (14782108, 2)
encoded: (14782108, 86172)
fitting...
done


Оценим качество предсказания на проверочной выборке, поскольку у нас бинарная классификация, используем log_loss

In [16]:
proba = base_model.predict_proba(X_test)
log_loss(y_test, proba[:,1])

encoding...
input X: (3761558, 2)
encoded: (3761558, 86172)
predicting...
done


0.7340844561133483

Построим рейтинг игроков

In [17]:
rating = base_model.player_rating() \
    .to_frame() \
    .join(players, how='inner', on='playerId') \
    .reset_index()
rating.head(20)

,playerId,playerRating,playerName
0,27403,13765.363048,Руссо Максим Михайлович
1,4270,13731.688312,Брутер Александра Владимировна
2,28751,13718.980223,Семушин Иван Николаевич
3,27822,13709.918048,Савченков Михаил Владимирович
4,30270,13681.990406,Спешков Сергей Леонидович
5,30152,13680.211915,Сорожкин Артём Сергеевич
6,87637,13635.776149,Саксонов Антон Владимирович
7,18036,13635.461137,Левандовский Михаил Ильич
8,20691,13633.587105,Мереминский Станислав Григорьевич
9,26089,13619.249173,Прокофьева Ирина Сергеевна


Сравним с рейтингом ЧГК по состоянию на 2 января 2020 года

https://rating.chgk.info/players.php?release=1430

In [18]:
top1000_2020 = pd.read_csv('./data/players-release-2020-01-02.csv')
rating20_20 = rating.head(20) \
    .merge(top1000_2020.head(20),
           how='left',
           left_on='playerId',
           right_on=top1000_2020.columns[0]
    )
display(rating20_20)
count20_20 = rating20_20[rating20_20[top1000_2020.columns[0]].notna()]
display(f'Количество игроков rating20 / топ20: {count20_20.shape[0]}')

,playerId,playerRating,playerName,ИД,Имя,Отчество,Фамилия,ИД базовой команды,Базовая команда,Место,Рейтинг
0,27403,13765.363048,Руссо Максим Михайлович,27403.0,Максим,Михайлович,Руссо,NaN,"Москва, Хельсинки, Санкт-Петербург, Самара, Мо...",5.0,14168.0
1,4270,13731.688312,Брутер Александра Владимировна,4270.0,Александра,Владимировна,Брутер,NaN,"Москва, Долгопрудный, Санкт-Петербург, Могилёв...",6.0,14166.0
2,28751,13718.980223,Семушин Иван Николаевич,28751.0,Иван,Николаевич,Семушин,NaN,"Москва, Долгопрудный, Санкт-Петербург, Киров, ...",2.0,14761.0
3,27822,13709.918048,Савченков Михаил Владимирович,27822.0,Михаил,Владимирович,Савченков,NaN,"Москва, Могилёв, Серпухов, Минск, Калининград,...",3.0,14747.0
4,30270,13681.990406,Спешков Сергей Леонидович,30270.0,Сергей,Леонидович,Спешков,NaN,"Москва, Пермь, Самара, Челябинск, Могилёв, Мин...",4.0,14708.0
5,30152,13680.211915,Сорожкин Артём Сергеевич,30152.0,Артём,Сергеевич,Сорожкин,NaN,"Москва, Долгопрудный, Санкт-Петербург, Калуга,...",1.0,14848.0
6,87637,13635.776149,Саксонов Антон Владимирович,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,18036,13635.461137,Левандовский Михаил Ильич,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,20691,13633.587105,Мереминский Станислав Григорьевич,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,26089,13619.249173,Прокофьева Ирина Сергеевна,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


'Количество игроков rating20 / топ20: 9'

In [19]:
rating100_top100 = rating.head(100) \
    .merge(top1000_2020.head(100),
           how='left',
           left_on='playerId',
           right_on=top1000_2020.columns[0]
    )
display(rating100_top100)
count100_100 = rating100_top100[rating100_top100[top1000_2020.columns[0]].notna()]
display(f'Количество игроков rating100 / топ100: {count100_100.shape[0]}')

,playerId,playerRating,playerName,ИД,Имя,Отчество,Фамилия,ИД базовой команды,Базовая команда,Место,Рейтинг
0,27403,13765.363048,Руссо Максим Михайлович,27403.0,Максим,Михайлович,Руссо,NaN,"Москва, Хельсинки, Санкт-Петербург, Самара, Мо...",5.0,14168.0
1,4270,13731.688312,Брутер Александра Владимировна,4270.0,Александра,Владимировна,Брутер,NaN,"Москва, Долгопрудный, Санкт-Петербург, Могилёв...",6.0,14166.0
2,28751,13718.980223,Семушин Иван Николаевич,28751.0,Иван,Николаевич,Семушин,NaN,"Москва, Долгопрудный, Санкт-Петербург, Киров, ...",2.0,14761.0
3,27822,13709.918048,Савченков Михаил Владимирович,27822.0,Михаил,Владимирович,Савченков,NaN,"Москва, Могилёв, Серпухов, Минск, Калининград,...",3.0,14747.0
4,30270,13681.990406,Спешков Сергей Леонидович,30270.0,Сергей,Леонидович,Спешков,NaN,"Москва, Пермь, Самара, Челябинск, Могилёв, Мин...",4.0,14708.0
...,...,...,...,...,...,...,...,...,...,...,...
95,90915,13415.511150,Яковлев Вадим Игоревич,90915.0,Вадим,Игоревич,Яковлев,46381.0,Разведка боём (Москва),20.0,13707.0
96,5328,13415.167097,Великов Дмитрий Вадимович,5328.0,Дмитрий,Вадимович,Великов,46381.0,Разведка боём (Москва),67.0,13048.0
97,72211,13414.356123,Подрядчикова Мария Владимировна,72211.0,Мария,Владимировна,Подрядчикова,46381.0,Разведка боём (Москва),57.0,13072.0
98,63529,13411.515329,Тарарыков Дмитрий Фёдорович,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


'Количество игроков rating100 / топ100: 45'

Baseline модель дала хороший результат, особенно в отношении первой пятерки игроков

# Оценка качества предсказания

3.Качество рейтинг-системы оценивается качеством предсказанийрезультатов турниров.

Предложите способ предсказать результаты нового турнирас известнымисоставами, но неизвестными вопросами, в виде ранжирования команд.

- считаем, что игроки дают или дают ответ на вопросы независимо друг от друга
- команда не может ответить на вопрос, когда ни один из игроков не может ответить на него

Вероятность команды ответить на вопрос будем считать через произведение вероятностей каждого игрока не ответить на вопрос:

$p_{team} = 1 - \prod (1 - p_{player})$.

In [20]:
test['proba'] = 1 - proba[:,1]
if 'playerRating' in test.columns:
    test = test.drop(columns=['playerRating'])

test['playerRating'] = test.merge(rating, how='left')['playerRating']

# средний рейтинг по команде
mean_rating = test.groupby(by=['tournamentId','teamId']) \
    .mean()['playerRating'] \
    .reset_index() \
    .rename(columns={'playerRating': 'meanRating'})

test.loc[test['playerRating'].isna(),['playerRating']] = \
    test[test['playerRating'].isna()].merge(mean_rating)['meanRating']

# средний рейтинг по турниру
mean_rating = test.groupby(by=['tournamentId']) \
    .mean()['playerRating'] \
    .reset_index() \
    .rename(columns={'playerRating': 'meanRating'})

test.loc[test['playerRating'].isna(),['playerRating']] = \
    test[test['playerRating'].isna()].merge(mean_rating)['meanRating']

test.describe().T

,count,mean,std,min,25%,50%,75%,max
tournamentId,3761558.0,5976.269948,214.998410,5414.000000,5754.000000,5962.000000,6184.000000,6456.000000
teamId,3761558.0,53737.397198,23430.710425,2.000000,46022.000000,60015.000000,72545.000000,78489.000000
position,3761558.0,172.224804,215.921788,1.000000,27.500000,87.000000,227.000000,1109.500000
playerId,3761558.0,124024.192585,70843.165037,15.000000,59399.000000,133028.000000,190276.000000,224697.000000
answer,3761558.0,0.463464,0.498663,0.000000,0.000000,0.000000,1.000000,1.000000
proba,3761558.0,0.628202,0.211824,0.077835,0.459111,0.634824,0.831982,0.995697
playerRating,3761558.0,8077.703937,3226.337903,322.204835,5665.315865,8486.695837,10924.617396,13731.688312


Посчитаем корреляцию по каждой команде, усредненную по всем турнирам

In [21]:
team_rating = test[['tournamentId', 'teamId', 'questionId', 'proba']] \
    .groupby(by=['tournamentId', 'teamId', 'questionId']) \
    .prod() \
    .groupby(by=['tournamentId', 'teamId']) \
    .mean() \
    .reset_index()

# чем больше команда отвечает на вопросы, тем ближе ее позиция в 1 месту, но больше рейтинг
#team_rating['proba'] = 1 - team_rating['proba']

team_rating['rating'] = team_rating.groupby('tournamentId')['proba'].rank()
team_rating['position'] = team_rating.merge(team_results, on=['tournamentId', 'teamId'])['position']

scores = team_rating[['tournamentId', 'rating', 'position']] \
    .groupby(by=['tournamentId']) \
    .agg(list)

scores['spearman_t'] = scores.apply(lambda x: spearmanr(x['position'], x['rating'])[0], axis=1)
scores['kendall_tau'] = scores.apply(lambda x: kendalltau(x['position'], x['rating'])[0], axis=1)
scores[['spearman_t', 'kendall_tau']].mean()

spearman_t     0.768320
kendall_tau    0.611975
dtype: float64

Корреляция Спирменана порядка 0.7-0.8,а корреляция Кендалла — порядка 0.5-0.6 - все так